<a href="https://colab.research.google.com/github/Junior-sls/Arquivos/blob/main/Videogamesearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive

In [16]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import pandas as pd

In [18]:
Data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Video Games Sales.csv')

In [19]:
display(pd.DataFrame(Data))

,index,Rank,Game Title,Platform,Year,Genre,Publisher,North America,Europe,Japan,Rest of World,Global,Review
0,0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,40.43,28.39,3.77,8.54,81.12,76.28
1,1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,91.00
2,2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,14.50,12.22,3.63,3.21,33.55,82.07
3,3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,14.82,10.51,3.18,3.01,31.52,82.65
4,4,5,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26,88.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,1902,1903,Lizzie McGuire 2: Lizzie Diaries,GBA,2004.0,Action,Disney Interactive Studios,0.60,0.22,0.00,0.01,0.83,55.00
1903,1903,1904,Xenoblade Chronicles,Wii,2010.0,Role-Playing,Nintendo,0.39,0.22,0.16,0.07,0.83,91.74
1904,1904,1905,SingStar Abba,PS3,2008.0,Misc,Sony Computer Entertainment,0.25,0.44,0.00,0.14,0.83,73.00
1905,1905,1906,FIFA Soccer World Championship,PS2,2000.0,Sports,Electronic Arts,0.27,0.21,0.28,0.07,0.83,73.00


In [20]:
!pip install nltk

In [21]:
import nltk

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
game_data = Data[['Game Title', 'Genre']].apply(lambda x: ' '.join(x.astype(str)), axis=1).tolist()
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(game_data)

In [25]:
def search_games(query, top_k=5):
    global Data
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, tfidf_matrix)
    results_indices = similarities.argsort()[0][::-1]
    results = []
    seen_games = set()
    for i, index in enumerate(results_indices, 1):
        row = Data.iloc[index]
        game_name = row['Game Title']
        platform = row['Platform']
        if (game_name, platform) not in seen_games:
            seen_games.add((game_name, platform))
            game_info = {
                'Name': game_name,
                'Publisher': row['Publisher'],
                'Year': row['Year'],
                'Platform': row['Platform'],
                'Rating': row['Review'],
                'Genre': row['Genre'],
                'Rank': i
            }
            results.append(game_info)
            if len(results) == top_k:
                break
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by=['Year', 'Rating'], ascending=[False, False])
    results_df.index = results_df.index + 1
    return results_df

In [29]:
query = input("Enter your Game: ")
results = search_games(query)
if results.empty:
    print("No games found.")
else:
    results_df = pd.DataFrame(results)
    results_df = results_df.reset_index(drop=True)
    results_df.index = results_df.index + 1
    display(results_df)
    print(results_df)

Enter your Game: Super Mario


,Name,Publisher,Year,Platform,Rating,Genre,Rank
1,Super Mario Advance 4: Super Mario Bros. 3,Nintendo,2003.0,GBA,90.0,Platform,2
2,Super Mario World: Super Mario Advance 2,Nintendo,2001.0,GBA,93.0,Platform,1
3,Super Mario World,Nintendo,1990.0,SNES,94.0,Platform,3
4,Super Mario Land,Nintendo,1989.0,GB,82.0,Platform,4
5,Super Mario Bros. 2,Nintendo,1988.0,NES,82.0,Platform,5


                                         Name Publisher    Year Platform  \
1  Super Mario Advance 4: Super Mario Bros. 3  Nintendo  2003.0      GBA   
2    Super Mario World: Super Mario Advance 2  Nintendo  2001.0      GBA   
3                           Super Mario World  Nintendo  1990.0     SNES   
4                            Super Mario Land  Nintendo  1989.0       GB   
5                         Super Mario Bros. 2  Nintendo  1988.0      NES   

   Rating     Genre  Rank  
1    90.0  Platform     2  
2    93.0  Platform     1  
3    94.0  Platform     3  
4    82.0  Platform     4  
5    82.0  Platform     5  
